# Homework 1
***

We are going to work with the following dataset: fluid current in a tube.
Some statistics are collected for dataset, incl. mean, skewness, kurtosis, etc. We are predicting flow rate ('tohn/hour'). We need to build confidence and predictive intervals.

In [16]:
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model, preprocessing, model_selection
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats as st
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [4]:
df = pd.read_csv('exxsol_data.csv', sep=';', header=(0))

There are 10 features and 1 label to predict:

In [6]:
print(df.columns.values)

['mean' 'std' 'skew' 'kurt' 'RMS' 'crest' 'freq_peak' 'shan' 'perm' 'temp'
 'tohn/hour']


In [8]:
y = df['tohn/hour']
freq_temp = df[['freq_peak','temp']]

Physics tells us that flow rate is a function of a frequency peak and temperature.

In [10]:
freq_temp, y = shuffle(freq_temp, y)

# split data into training and testing sets
#from sklearn.model_selection import train_test_split
#train_freq, test_freq, train_y, test_y = train_test_split(freq, y, train_size=0.7, random_state=2)

lr = linear_model.LinearRegression()
predicted = model_selection.cross_val_predict(
    lr, freq_temp, y.ravel(), cv=20)
score = model_selection.cross_val_score(lr, freq_temp, y,
                                         scoring='r2',cv=20)

C:\Users\lplem\AppData\Local\Temp\ipykernel_14476\2340299872.py:9: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  lr, freq_temp, y.ravel(), cv=20)


## Q0: Build point estimate for mean r2 score and its deviation

In [18]:
# Your code here
mean = np.mean(score)
var = np.var(score, ddof=1)
_tconfint_generic(mean, np.sqrt(var), len(score) - 1, 0.05, 'two-sided')

(0.6145638645717862, 1.0350965234848906)

## Q1: Predicted is an array with predictions of the label y. Assuming, that $\sigma = 0.1$, compute 95% confidence and predictive interval for mean squared error. 

In [28]:
# Your code here
error = np.sqrt((y - predicted)**2)  
error_mean = np.mean(error)
sigma = 0.1
n = len(error)
confidence_interval = _zconfint_generic(error_mean, 0.1, 0.05, 'two-sided')
predictive_interval = _zconfint_generic(error_mean, sigma*(1+1/n), 0.05, 'two-sided')

print("confidence_interval: ", confidence_interval)
print("predictive_interval: ", predictive_interval)

confidence_interval:  (0.12568539570809995, 0.5176781926161107)
predictive_interval:  (0.12540540085316565, 0.517958187471045)


## Q2:  Compute 95% confidence and predicted intervals for mean squared error, assuming no knowledge about $\sigma$.

In [36]:
# Your code here
error = np.sqrt((y - predicted)**2)  
error_mean = np.mean(error)
error_var = np.var(error, ddof=1)
n = len(error)
confidence_interval = _tconfint_generic(error_mean, np.sqrt(error_var), n-1, 0.05, 'two-sided')
predictive_interval = _tconfint_generic(error_mean, np.sqrt(error_var)*(n+1)/n, n-1, 0.05, 'two-sided')

print("confidence_interval: ", confidence_interval)
print("predictive_interval: ", predictive_interval)

confidence_interval:  (-0.3978931427832453, 2.5965988573483685)
predictive_interval:  (-0.4000320656404821, 2.5987377802056058)


We can use additional features and more complex model, e.g. ElasticNet.

In [34]:
y = df['tohn/hour']
X = df.drop(['tohn/hour'],axis=1)
X = preprocessing.scale(X)
X, y = shuffle(X, y)

encv = linear_model.ElasticNetCV(cv=10,max_iter=3000, n_alphas=10)
predicted_encv = model_selection.cross_val_predict(
    encv, X, y.ravel(), cv=20)
score_encv = model_selection.cross_val_score(encv,X, y.ravel(),
                                         scoring='r2',cv=20)

C:\Users\lplem\AppData\Local\Temp\ipykernel_14476\3988676496.py:8: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  encv, X, y.ravel(), cv=20)
C:\Users\lplem\AppData\Local\Temp\ipykernel_14476\3988676496.py:9: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  score_encv = model_selection.cross_val_score(encv,X, y.ravel(),


## Q3:  Compute 95% confidence interval for difference in means of mean squared error between 2 models, assuming no knowledge about $\sigma$.

In [48]:
# Your code here
error = np.sqrt((y - predicted)**2) 
error_encv = np.sqrt((y - predicted_encv)**2) 
error_mean = np.mean(error)
error_encv_mean = np.mean(error_encv)
n = len(error)
error_var = np.var(error, ddof=1)
error_encv_var = np.var(error_encv, ddof=1)
var = (error_var + error_encv_var) / 2


_tconfint_generic(error_mean - error_encv_mean, np.sqrt(var), 2*(n - 1), 0.05, 'two-sided')

(-0.1564679893024966, 2.01370364874248)

## Q4: Implement UCB1